In [14]:
import math
import numpy
import itertools
import random

In [15]:
def stump_functions():
    s = [1,-1]
    b = [-2.0,-0.5,0.5,2.0]
    d = [1,2]
    return itertools.product(s,b,d)

def input_data():
    X = [ [0.0, math.sqrt(2)], [-math.sqrt(2), 0.0], [0.0,-math.sqrt(2)], [math.sqrt(2), 0.0] ]
    y = [1,-1,1,-1]
    return numpy.array(X), numpy.array(y)

def classify(x, stump_function):
    if x[stump_function[2]-1] > stump_function[1]:
        return stump_function[0]
    else:
        return -1 * stump_function[0]

def error(y, weights, classification_results):
    nd = weights.shape[0]
    weighted_error = 0.0
    for i in range(nd):
        if y[i] != classification_results[i]:
            weighted_error += weights[i]
    return weighted_error

In [16]:
X,y = input_data()
n,nd = X.shape
weights = numpy.full((n,),1.0/n)
T = 3
stumps = list(stump_functions())

In [17]:
betas = []
selected_stumps = []

In [18]:
for t in range(T):
    # Run for each T best stumps
    print "+++++++++++ t = "+str(t)+ " +++++++"
    min_error = float("inf")
    selected_stump = None
    
    # (a,b) Find the best stump (can choose same stump across iterations, mostly it won't happen)
    random.shuffle(stumps)
    for stump in stumps:
        classification_results = []
        for i in range(n):
            classification_results.append(classify(X[i], stump))
        weighted_error = error(y, weights, classification_results)
        if weighted_error <= min_error:
            min_error = weighted_error
            selected_stump = stump
    
    print("error",min_error, "chosen stump", selected_stump)
    
    # (c) Calculate beta
    beta = 0.5 * math.log( (1 - min_error) / min_error)
    print ("beta",beta)
    print ("weights old",weights)
    
    # for future use
    betas.append(beta)
    selected_stumps.append(selected_stump)
    
    # (d) Update weights for next iteration
    for i in range(n):
        classification_chosen_stump = classify(X[i], selected_stump)
        if y[i] == classification_chosen_stump:
            weights[i] = weights[i]*math.exp(-beta)
        else:
            weights[i] = weights[i]*math.exp(beta)
    # (e) Calculate norms
    norm = sum(weights)
    weights = weights/norm
    
    print ("weights new",weights)

+++++++++++ t = 0 +++++++
('error', 0.25, 'chosen stump', (-1, -0.5, 2))
('beta', 0.5493061443340549)
('weights old', array([ 0.25,  0.25,  0.25,  0.25]))
('weights new', array([ 0.5       ,  0.16666667,  0.16666667,  0.16666667]))
+++++++++++ t = 1 +++++++
('error', 0.16666666666666666, 'chosen stump', (1, 0.5, 2))
('beta', 0.8047189562170503)
('weights old', array([ 0.5       ,  0.16666667,  0.16666667,  0.16666667]))
('weights new', array([ 0.3,  0.1,  0.5,  0.1]))
+++++++++++ t = 2 +++++++
('error', 0.099999999999999964, 'chosen stump', (-1, 0.5, 1))
('beta', 1.0986122886681098)
('weights old', array([ 0.3,  0.1,  0.5,  0.1]))
('weights new', array([ 0.16666667,  0.5       ,  0.27777778,  0.05555556]))


In [19]:
correct = 0
for i in range(n):
    main_classifier_val = 0.0
    for t in range(T):
        classification = classify(X[i], selected_stumps[t])
        main_classifier_val += betas[t] * classification
    
    classification = numpy.sign(main_classifier_val)
    print ("X: ", X[i], classification)
    
    if y[i] == classification:
        correct += 1

print ("correct: ", correct)

('X: ', array([ 0.        ,  1.41421356]), 1.0)
('X: ', array([-1.41421356,  0.        ]), -1.0)
('X: ', array([ 0.        , -1.41421356]), 1.0)
('X: ', array([ 1.41421356,  0.        ]), -1.0)
('correct: ', 4)
